In [35]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
# font 설정
import platform
path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else :
    print('Unknown system... sorry~')

In [119]:
train =  pd.read_csv("bike-sharing-demand/train.csv",parse_dates=["datetime"])
test = pd.read_csv("bike-sharing-demand/test.csv",parse_dates=["datetime"])

In [120]:
train['year'] =train['datetime'].dt.year
train['month'] =train['datetime'].dt.month
train['hour'] =train['datetime'].dt.hour
train['dayofweek'] =train['datetime'].dt.dayofweek

In [121]:
test['year'] =test['datetime'].dt.year
test['month'] =test['datetime'].dt.month
test['hour'] =test['datetime'].dt.hour
test['dayofweek'] =test['datetime'].dt.dayofweek

In [122]:
#이상치 제거
train_out =  train[np.abs(train['count']-train['count'].mean())<=(3*train['count'].std())]

In [123]:
print(train.shape)
print(train_out.shape)

(10886, 16)
(10739, 16)


In [124]:
#windspeed 값 채우기
train =  pd.read_csv("bike-sharing-demand/train.csv",parse_dates=["datetime"])
test = pd.read_csv("bike-sharing-demand/test.csv",parse_dates=["datetime"])

In [125]:
#train, test 합치기
data = train.append(test)
data.reset_index(inplace= True)
data.drop('index', inplace=True, axis=1)

In [126]:
data['year'] =data['datetime'].dt.year
data['month'] =data['datetime'].dt.month
data['hour'] =data['datetime'].dt.hour
data['dayofweek'] =data['datetime'].dt.dayofweek


In [127]:
#랜덤 포레스트로 windspeed 채우기
dataWind0 = data[data['windspeed']==0]
dataWind1 = data[data['windspeed']!=0]
#모델 객체 생성, 사용할 feature(windCol), fit하기 
rfModel_wind = RandomForestRegressor()
windCol= ['season', 'weather','humidity', 'month', 'year', 'temp','atemp']
rfModel_wind.fit(dataWind1[windCol], dataWind1['windspeed'])
#예측, 대입 
wind0Values = rfModel_wind.predict(X = dataWind0[windCol])
dataWind0['windspeed']=wind0Values

data = dataWind1.append(dataWind0)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)


In [128]:
dropFeatures = ['casual',"count","registered"]

In [129]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])


In [131]:
label = dataTrain['count']
xData = dataTrain.drop(['datetime','casual','registered','count'],axis=1)
test = dataTest.drop(['datetime','casual','registered','count'],axis=1)

In [132]:
rfModel = RandomForestRegressor()
rfModel.fit(xData, label)
prediction = rfModel.predict(test)
# res = pd.DataFrame({'datetime':datetime_,'count':prediction})

In [133]:
res.to_csv("result4.csv",index=False)
pd.read_csv("result4.csv")

,datetime,count
0,2011-01-20 00:00:00,165.786000
1,2011-01-20 01:00:00,54.903000
2,2011-01-20 02:00:00,54.903000
3,2011-01-20 03:00:00,98.720000
4,2011-01-20 04:00:00,98.720000
...,...,...
6488,2012-12-31 19:00:00,124.290190
6489,2012-12-31 20:00:00,124.290190
6490,2012-12-31 21:00:00,73.471667
6491,2012-12-31 22:00:00,58.726667
